In [10]:
import os
from urllib import request
from imgurpython import ImgurClient
import timeit
import time
import requests
import json
import Lab10_multiprocessing_part2 as multiprocessing_file
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
import nest_asyncio
nest_asyncio.apply()
import asyncio
import aiohttp

In [3]:
def download_images():
    response = requests.get("https://picsum.photos/v2/list")
    if response.status_code != 200:
        raise AttributeError('GET /tasks/ {}'.format(response.status_code))
    data = json.loads(response.text)

    pictures=[]
    for s in data:
        pictures.append(s['download_url']+".jpg")
    return pictures

def saveImages(link):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    request.urlretrieve(link, "downloads/{}.{}".format(filename, fileformat))

def main():
    images = download_images()
    for image in images:
        saveImages(image)

start_time = time.time()
main()
duration_synch = time.time() - start_time
print(f"Time taken to download 30 images into the downloads folder synchronously: {duration_synch}")

Time taken to download 30 images into the downloads folder synchronously: 32.752798557281494


In [19]:
def process_images_threading():
    images = download_images()
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(saveImages,images)

start_time = time.time()
process_images_threading()
duration_threading = time.time() - start_time
print(f"Time taken to download 30 images into the downloads folder with multithreading: {duration_threading}")

Time taken to download 30 images into the downloads folder with multithreading: 7.226224422454834


In [4]:
def process_images_multiprocessing():
    images = download_images()
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    pool.map(multiprocessing_file.multiprocessing_images,images)


start_time = time.time()
process_images_multiprocessing()
duration_multiprocessing = time.time() - start_time
print(f"Time taken to download 30 images into the downloads folder with multiprocessing: {duration_multiprocessing}")

Time taken to download 30 images into the downloads folder with multiprocessing: 9.249184131622314


In [13]:
async def download_images_asyncio(link, session):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    async with session.get(link) as response:
        with open("downloads/{}.{}".format(filename, fileformat), 'wb') as fd:
            async for data in response.content.iter_chunked(1024):
                fd.write(data)

async def main_asyncio():
    images = download_images()

    async with aiohttp.ClientSession() as session:
        tasks=[download_images_asyncio(image,session)for image in images]
        return await asyncio.gather(*tasks)

start_time = time.time()
asyncio.run(main_asyncio())
duration_asyncio = time.time() - start_time
print(f"Time taken to download 30 images into the downloads folder with asyncio: {duration_asyncio}")

Time taken to download 30 images into the downloads folder with asyncio: 3.49655818939209
